In [ ]:
### IMPORTS ###
import pandas as pd
import numpy as np
import pathlib as pl
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import classification_report

!pip install datasets
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

!pip install transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler
from transformers import TrainingArguments

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

!pip install evaluate
import evaluate

from transformers import Trainer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PATH = "/content/drive/My Drive/PLN/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
### CONJUNTOS DE ENTRENAMIENTO, VALIDACIÓN Y TEST ###
df = pd.read_csv(PATH + "train.csv",  sep=',', on_bad_lines='skip', encoding='utf-8', encoding_errors='ignore')
df = df[['tweet', 'mean_prejudice']]
df = df.rename(columns={"tweet": "Text", "mean_prejudice": "Label"})
df.fillna(" ", inplace=True)

X_train = df['Text']
y_train = df['Label']

X_train, X_aux, y_train, y_aux = train_test_split(X_train, y_train, test_size=0.3, random_state=55)
X_val, X_test, y_val, y_test = train_test_split(X_aux, y_aux, test_size=0.66, random_state=55)

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

print('Tamaño conjunto de Entrenamiento:', len(X_train))
print('Tamaño conjunto de Validación:', len(X_val))
print('Tamaño conjunto de Evaluación:', len(X_test))

d = {'train':Dataset.from_dict({'Label':y_train, 'Text':X_train}),
     'val':Dataset.from_dict({'Label':y_val, 'Text':X_val}),
     'test':Dataset.from_dict({'Label':y_test, 'Text':X_test})
     }

dict_dataset = DatasetDict(d)

Tamaño conjunto de Entrenamiento: 1869
Tamaño conjunto de Validación: 272
Tamaño conjunto de Evaluación: 530


# Tokenization

In [ ]:
model_name ='bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

MAX_LENGTH = 55

In [ ]:
def tokenize(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)


data_encodings = dict_dataset.map(tokenize, batched=True)
data_encodings

Map:   0%|          | 0/1869 [00:00<?, ? examples/s]

Map:   0%|          | 0/272 [00:00<?, ? examples/s]

Map:   0%|          | 0/530 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Label', 'Text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1869
    })
    val: Dataset({
        features: ['Label', 'Text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 272
    })
    test: Dataset({
        features: ['Label', 'Text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 530
    })
})

# Modelo

Necesitamos hacer algunas modificaciones para preparar el dataset para el modelo:

- Eliminamos la columna 'text', porque es un campo que el modelo no espera.
- Renombramos 'label' a 'labels', porque es el nombre que espera el modelo.
- El dataset debe devolver un objeto Tensor en lugar de una lista.

Para pasarle los datos al modelo debemos guardarlos en objetos DataLoader

In [ ]:
data_encodings = data_encodings.remove_columns('Text')
data_encodings = data_encodings.rename_column('Label', 'labels')
data_encodings.set_format("torch")
data_encodings

train_dataloader = DataLoader(data_encodings['train'], shuffle=True, batch_size=8)
val_dataloader = DataLoader(data_encodings['val'], batch_size=8)

Para definir el modelo, hay que establecer el número de etiquetas:

In [ ]:
NUM_LABELS = 1
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS) 

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Como optimizador vamos a utilizar el optijizador Adam, implementado en AdamW, y definimos el learning rate:

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

# Hyperparámetros

In [ ]:
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate

In [ ]:
args = TrainingArguments(output_dir="./outputs")
args.evaluation_strategy="epoch"
args.per_device_train_batch_size = 32
args.per_device_eval_batch_size = 32
args.num_train_epochs = 5

# Métrica

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error
from scipy import stats

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    # loss metrics
    mse = mean_squared_error(labels, logits)
    rmse = mean_squared_error(labels, logits, squared=False)
    mae = mean_absolute_error(labels, logits)
    smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)
    # performance metrics
    r2 = r2_score(labels, logits)
    # pearson=stats.pearsonr(labels, logits)[0]

    # we return a dictionary with all metrics
    # return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "smape": smape, "pearson": pearson}
    return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "smape": smape}

# Training

In [ ]:
trainer = Trainer(
    model=model,            # modelo que será ajustado
    args = args,     # hiperparámetros
    train_dataset=data_encodings['train'], # conjunto training
    eval_dataset=data_encodings['val'],   # conjunto de validación
    compute_metrics=compute_metrics,    # función para computar las métricas
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mse,Rmse,Mae,R2,Smape
1,No log,0.510525,0.510525,0.714510,0.569826,0.139576,20.068465
2,No log,0.618298,0.618298,0.786320,0.622640,-0.042062,21.408514
3,No log,0.671605,0.671605,0.819515,0.651992,-0.131903,22.390232
4,No log,0.549772,0.549772,0.741466,0.581991,0.073430,20.329525
5,No log,0.561323,0.561323,0.749215,0.588083,0.053963,20.410291


TrainOutput(global_step=295, training_loss=0.44240257133871824, metrics={'train_runtime': 107.3028, 'train_samples_per_second': 87.09, 'train_steps_per_second': 2.749, 'total_flos': 264123614219850.0, 'train_loss': 0.44240257133871824, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.5613230466842651,
 'eval_mse': 0.5613230466842651,
 'eval_rmse': 0.7492149472236633,
 'eval_mae': 0.5880830883979797,
 'eval_r2': 0.0539625073054143,
 'eval_smape': 20.41029088637408,
 'eval_runtime': 0.8785,
 'eval_samples_per_second': 309.622,
 'eval_steps_per_second': 10.245,
 'epoch': 5.0}

# Evaluación

In [ ]:
def get_prediction(text):
    inputs = tokenizer(text, max_length=MAX_LENGTH, padding="max_length", truncation=True, return_tensors="pt").to("cuda")
    outputs = model(**inputs)   
    return outputs[0].item()   

y_pred=[get_prediction(text) for text in dict_dataset['test']['Text']]

In [ ]:
def print_metrics(y_test, y_pred, lang=''):
    if lang:
        pass
    else:
        #         print("Final results on the whole test dataset")
        print('|   |MSE|RMSE|MAE|R2|SMAPE|PEARSON|')
        print('|---|---|---|---|---|---|---|')

    mse = mean_squared_error(y_test, y_pred)
    # print ("MSE: ", "{:.2f}".format(mse), end=', ')

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    # print("RMSE: ", "{:.2f}".format(rmse), end=', ')
    
    mae = mean_absolute_error(y_test, y_pred)
    # print("MAE: ", "{:.2f}".format(mae), end=', ')

    r2 = r2_score(y_test, y_pred)
    # print("R2: ", "{:.2f}".format(r2), end=', ')

    diff=[label-pred for (label,pred) in zip(y_test,y_pred)]
    smape= 1/len(y_test) * np.sum(2 * np.abs(diff) / (np.abs(y_test) + np.abs(y_pred))*100)
    # print("SMAPE: ", "{:.2f}".format(smape), end=', ')

    pearson=stats.pearsonr(y_test, y_pred)[0]
    # print("PEARSON: ", "{:.2f}".format(pearson))
    # print()

    text_table=str('|')+lang+str('|') +"{:.2f}".format(mse) \
                +str('|')+"{:.2f}".format(rmse) \
                +str('|')+"{:.2f}".format(mae) \
                +str('|')+"{:.2f}".format(r2) \
                +str('|')+"{:.2f}".format(smape) \
                +str('|')+"{:.2f}".format(pearson) + str('|')

    print(text_table)

In [ ]:
print_metrics(y_test, y_pred)

|   |MSE|RMSE|MAE|R2|SMAPE|PEARSON|
|---|---|---|---|---|---|---|
||0.58|0.76|0.59|0.13|20.23|0.49|
